# 1. Importing important libraries

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import timeit

import os
import pickle
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import  confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from matplotlib.pyplot import figure
import wandb

wandb.login()

/home/anwer/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: danish2562022. Use `wandb login --relogin` to force relogin


True

# 2. Basic EDA

In [56]:
# train = pd.read_csv("../../csv_files_new_ppi/training_and_test_set/train_set_without_embedding.csv")
test =  pd.read_csv("../../csv_files_new_ppi/training_and_test_set/test_set_without_embedding.csv")

In [57]:
# train_pos = train[train.label == 1][0:100000]
# train_neg = train[train.label == 0][0:100000]
# train = pd.concat([train_pos,train_neg])

test_pos = test[test.label == 1]
test_neg = test[test.label ==0][0:len(test_pos)*5]
test = pd.concat([test_pos,test_neg])
test.label.value_counts()

0    106735
1     21347
Name: label, dtype: int64

In [58]:
val_pos =  test[test.label == 1][0:10000]
val_neg =  test[test.label == 0][0:10000]
val = pd.concat([val_pos,val_neg])

test_pos =  test[test.label == 1][10000:]
test_neg =  test[test.label == 0][10000:]
test = pd.concat([test_pos,test_neg])

In [59]:
# print("Size of Train dataset: ", len(train))
print("Size of Test dataset: ", len(test))
print("Size of val dataset: ", len(val))

Size of Test dataset:  108082
Size of val dataset:  20000


In [77]:
# print(f"Number of negative points in training set: {train.label.value_counts()[0]}")
# print(f"Number of positive points in training set: {train.label.value_counts()[1]}")
print("----"*57)
print(f"Number of negative points in test set: {test.label.value_counts()[0]}")
print(f"Number of positive points in test set: {test.label.value_counts()[1]}")
print("----"*57)
print(f"Number of negative points in val set: {val.label.value_counts()[0]}")
print(f"Number of positive points in val set: {val.label.value_counts()[1]}")

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Number of negative points in test set: 96735
Number of positive points in test set: 11347
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Number of negative points in val set: 10000
Number of positive points in val set: 10000


# 3. Importing embedding vectors from pickle file

In [62]:
with open("../../pickle/embedding_vectors_pickle/sum_of_amino_acid_vector.pickle",'rb') as handle:
    dc = pickle.load(handle)

In [63]:
def return_embed(prot_name):
    try:
        return dc[prot_name]
    except:
        return np.nan
# train['embed_vec_protein_A'] = train["Protein_A"].apply(return_embed)
# train['embed_vec_protein_B'] = train["Protein_B"].apply(return_embed)

test['embed_vec_protein_A'] = test["Protein_A"].apply(return_embed)
test['embed_vec_protein_B'] = test["Protein_B"].apply(return_embed)

# val['embed_vec_protein_A'] = val["Protein_A"].apply(return_embed)
# val['embed_vec_protein_B'] = val["Protein_B"].apply(return_embed)


# train = train.dropna()
test = test.dropna()
# val = val.dropna()

In [64]:
train_features_Protein_A = []
train_features_Protein_B = []
train_label = []
test_features_Protein_A = []
test_features_Protein_B = []
test_label =[]
val_features_Protein_A = []
val_features_Protein_B = []
# val_label = []
# for i in tqdm(range(len(train))):
#     train_features_Protein_A.append(np.array(train.iloc[i].embed_vec_protein_A))
#     train_features_Protein_B.append(np.array(train.iloc[i].embed_vec_protein_B))
#     train_label.append(np.array(train.iloc[i].label))
    
for i in tqdm(range(len(test))):
    test_features_Protein_A.append(np.array(test.iloc[i].embed_vec_protein_A))
    test_features_Protein_B.append(np.array(test.iloc[i].embed_vec_protein_B))
    test_label.append(np.array(test.iloc[i].label))  
# for i in tqdm(range(len(val))):
    
    
#     val_features_Protein_A.append(np.array(val.iloc[i].embed_vec_protein_A))
#     val_features_Protein_B.append(np.array(val.iloc[i].embed_vec_protein_B))
#     val_label.append(np.array(val.iloc[i].label))

100%|████████████████████████████████████████████████| 108082/108082 [00:33<00:00, 3204.22it/s]


In [65]:
# train_features_Protein_A = np.array(train_features_Protein_A)
# train_features_Protein_B = np.array(train_features_Protein_B)
# train_label = np.array(train_label)

test_features_Protein_A = np.array(test_features_Protein_A)
test_features_Protein_B = np.array(test_features_Protein_B)
test_label = np.array(test_label)

# val_features_Protein_A = np.array(val_features_Protein_A)
# val_features_Protein_B = np.array(val_features_Protein_B)
# val_label = np.array(val_label)

# 4. Dataloader

In [66]:
class Data(Dataset):
    
    def __init__(self, X_data_A,X_data_B, y_data):
        self.X_data_A = X_data_A
        self.X_data_B = X_data_B
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data_A[index],self.X_data_B[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data_A)

In [67]:
# train_data = Data(torch.FloatTensor(train_features_Protein_A), torch.FloatTensor(train_features_Protein_B),
#                        torch.FloatTensor(train_label))

test_data = Data(torch.FloatTensor(test_features_Protein_A), torch.FloatTensor(test_features_Protein_B),
                       torch.FloatTensor(test_label))

# val_data = Data(torch.FloatTensor(val_features_Protein_A), torch.FloatTensor(val_features_Protein_B),
#                        torch.FloatTensor(val_label))

In [68]:

# train_loader = DataLoader(dataset=train_data, batch_size=512, shuffle=True,drop_last=True)
test_loader = DataLoader(dataset=test_data, batch_size=512,shuffle=True,drop_last=True )
# val_loader = DataLoader(dataset=val_data, batch_size=512,shuffle=True,drop_last=True )

In [69]:
for i,j,k in test_loader:
    print(i.shape)
    print(j.shape)
    print(k.shape)
    break
    

torch.Size([512, 1024])
torch.Size([512, 1024])
torch.Size([512])



# 5. Model_testing

### 5.1  Approach_2_concat_siamese_network_on_sum_(Probert-BFD)
<pre>
Training Script: Approach_2_concat_siamese_network_on_sum_.ipynb
</pre>


In [15]:
class BertClassifier(nn.Module):
    
    def __init__(self,config, embed_dim =1024):
        super(BertClassifier,self).__init__()
        self.relu = nn.ReLU()
        self.config = config
        self.conv1  = nn.Conv1d(in_channels = 1,out_channels = 33, kernel_size = 3, stride=1)
        self.fc1 = nn.Linear(33726,config.dim_1)
        self.fully_connected_layers_1 = nn.ModuleList([nn.Linear(config.dim_1*2,config.dim_1*2)
                                                    for _ in range(config.layer_fc_1)])
        
        
    
        self.fc_2 = nn.Linear(config.dim_1*2,config.dim_2)
        self.fully_connected_layers_2 = nn.ModuleList([nn.Linear(config.dim_2,config.dim_2)
                                                    for _ in range(config.layer_fc_2)]) 
        self.bn2 = nn.BatchNorm1d(num_features=config.dim_2)
        self.fc3 = nn.Linear(config.dim_2,config.dim_1)
        
        self.fc4 = nn.Linear(config.dim_1,256)
        self.drop = nn.Dropout(p = 0.2)
        self.fc5 = nn.Linear(256,128)
        self.fc6 = nn.Linear(128,64)
        self.fc7 = nn.Linear(64,32)
        self.fc8 = nn.Linear(32,16)
        self.fc9 = nn.Linear(16,8)
        self.fc10 = nn.Linear(8,1)
    
    def forward(self, inputs_A,inputs_B):
        
        
        inputs_A = inputs_A.reshape(512,1,1024)
        output_conv_A = self.relu(self.conv1(inputs_A))
        output_conv_A = output_conv_A.reshape(512,33726) 
        output_A = self.relu(self.fc1(output_conv_A))
        
        inputs_B = inputs_B.reshape(512,1,1024)
        output_conv_B = self.relu(self.conv1(inputs_B))
        output_conv_B = output_conv_B.reshape(512,33726) 
        output_B = self.relu(self.fc1(output_conv_B))
        
        
        output = torch.cat((output_A, output_B),1)
        for i in range(self.config.layer_fc_1):
            output = self.relu(self.fully_connected_layers_1[i](output))
        output = self.relu(self.fc_2(output))
        for i in range(self.config.layer_fc_2):
            output = self.relu(self.fully_connected_layers_2[i](output))
            
        output  = self.bn2(output)
        output = self.relu(self.fc3(output))
        if self.config.dropout:
            output = self.drop(output)
            
        output = self.relu(self.fc4(output))
        output = self.relu(self.fc5(output))
        output = self.relu(self.fc6(output))
        output = self.fc7(output)
        output = self.fc8(output)
        output = self.fc9(output)
        output = self.fc10(output)

        return output

In [16]:
def evaluate(val_dataloader,config):
    device = "cpu"
    loss_fn = nn.BCEWithLogitsLoss()
    model = BertClassifier(config)
    model.load_state_dict(torch.load("best_model_trained_fc_v2.pth")['model_state_dict'])
    model.eval()
    val_accuracy = []
    val_loss = []
    for batch in tqdm(val_dataloader):
        device = torch.device("cpu")
        inputs_A,inputs_B ,b_labels = tuple(t.to(device) for t in batch)
        b_labels = b_labels.reshape((1,512,1)).squeeze(0)
        with torch.no_grad():
                logits = model(inputs_A,inputs_B)
        
        loss = loss_fn(logits, b_labels.float())
        val_loss.append(loss.item())
        preds = torch.round(torch.sigmoid(logits))
        
        accuracy = (preds.float() == b_labels.float()).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)
    
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy,model 

In [17]:
config = dict(layer_fc_1 = 2,
        dim_1 = 2048,
        layer_fc_2 = 2,
        dim_2 = 512,
        dropout = False,
        optimizer = 'adam',
        learning_rate = 0.00007,
        momentum = 0.95,
        weight_decay = 0.01,
        amsgrad = False,
        epochs = 500)
# model = BertClassifier(config)
config

{'layer_fc_1': 2,
 'dim_1': 2048,
 'layer_fc_2': 2,
 'dim_2': 512,
 'dropout': False,
 'optimizer': 'adam',
 'learning_rate': 7e-05,
 'momentum': 0.95,
 'weight_decay': 0.01,
 'amsgrad': False,
 'epochs': 500}

In [20]:
def model_pipeline(config=None):

    # tell wandb to get started
    with wandb.init(project="model_testing_Approach_2_concat_siamese_network_on_sum_", config=config):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config
      for i,j,k in test_loader:
          x = (i,j)
          
      val_loss,val_accuracy,model = evaluate(test_loader,config)
      
      
      torch.onnx.export(model,x, "model.onnx")
      wandb.save("model.onnx")
      return val_loss,val_accuracy,model

In [21]:
val_loss,val_accuracy,model = model_pipeline(config)

100%|██████████████████████████████████████████████████████████| 44/44 [01:32<00:00,  2.11s/it]


In [22]:
print("val_loss: ",val_loss)
print("val_accuracy: ",val_accuracy)

val_loss:  0.6291291036389091
val_accuracy:  76.26509232954545


In [70]:
truth = []
predicted = []   
for inputs_A,inputs_B,b_label in tqdm(test_loader):
    b_label = b_label.reshape((1,512,1)).squeeze(0)
    with torch.no_grad():
        logits = model(inputs_A,inputs_B)
        
    preds = torch.round(torch.sigmoid(logits))
    predicted.extend(preds.detach().numpy())
    truth.extend(b_label.detach().numpy())
    
        
    
    

100%|████████████████████████████████████████████████████████| 211/211 [07:28<00:00,  2.13s/it]


In [117]:
TN = 0
FN = 0  
TP = 0
FP = 0

for i,j in tqdm(enumerate(truth)):
    if int(j[0]) == 0:
        if int(predicted[i][0]) == 0:
            TN += 1
        else:
            FP += 1
            
    elif int(j[0]) == 1:
        if int(predicted[i][0]) == 0:
            FN += 1
        else:
            TP += 1
            

108032it [00:00, 757610.14it/s]


In [118]:
print(f"Positive Accuracy/Recall/Sensitivity/TPR: {TP/truth.count(1)*100}%")
print(f"Negative Accuracy/Specificity/TNR: {TN/truth.count(0)*100}%")

Positive Accuracy/Recall/Sensitivity/TPR: 71.73721340388008%
Negative Accuracy/Specificity/TNR: 80.76055930170024%


In [125]:
assert(FP+TN == truth.count(0))
assert(FN+TP == truth.count(1))

In [127]:
false_positive_rate = FP/truth.count(0)
false_negative_rate = FN/truth.count(1)
print(f"FPR: {false_positive_rate *100}%")
print(f"FNR: {false_negative_rate *100}%")

FPR: 19.239440698299756%
FNR: 28.26278659611993%


#### Results summary 

<pre>
Positive Accuracy: 71.7327%
negative_Accuracy: 80.76%
FPR: 19.24%
FNR: 28.26%
</pre>